In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,make_scorer
import shap
import os
import pickle

import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor

c:\Users\jacks\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
#import training and testing data
train = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Winter23\Sports Analytics\Passion Project\train2.csv")
test = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Winter23\Sports Analytics\Passion Project\test3.csv")
test_2023 = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Winter23\Sports Analytics\Passion Project\test2.csv")

In [55]:
#fill null value
train=train.fillna(0)
test=test.fillna(0)
test_2023 = test_2023.fillna(0)

In [56]:
train.WinPCT = train.WinPCT*100
test.WinPCT = test.WinPCT*100
test_2023.WinPCT = test_2023.WinPCT*100

In [57]:
encoded_train = pd.get_dummies(train, columns=['Pos'])
encoded_test = pd.get_dummies(test, columns=['Pos'])
encoded_test2023 = pd.get_dummies(test_2023, columns=['Pos'])

In [58]:
encoded_test2023 = encoded_test2023[encoded_test2023.G>=43]
encoded_test2023 = encoded_test2023[encoded_test2023.MP/encoded_test2023.G >=30]

In [65]:
encoded_train.columns

Index(['# Tm', 'Player', 'Age', 'Share', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL',
       'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'OBPM', 'DBPM', 'BPM', 'VORP', 'WinPCT', 'Tm', 'year', 'season',
       'SeasonID', 'Team', 'Pos_C', 'Pos_F', 'Pos_G'],
      dtype='object')

In [63]:
real_test2023.columns

Index(['PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'WinPCT', 'Pos_C', 'Pos_F', 'Pos_G'],
      dtype='object')

In [61]:
real_test = encoded_test.drop(columns=['# Tm','Share','Player','Age','G','MP'])

In [60]:
real_test2023 = encoded_test2023.drop(columns=['Team','Player','Age','G','MP'])

In [68]:
real_test.shape

(27, 32)

In [66]:
X = encoded_train.drop(columns=['# Tm','Player','Share','SeasonID','Team','Tm','season','year','Age','G','MP'])
y = encoded_train.Share

In [14]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
# create a random forest model
model = RandomForestRegressor(n_estimators=100)

# train the model
model.fit(X_train, y_train)

# make predictions on the test set
y_pred = model.predict(X_test)

# evaluate the model using RMSE, R-squared, and MAE
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('RMSE:', rmse)
print('R-squared:', r2)
print('MAE:', mae)

RMSE: 0.15280751886249758
R-squared: 0.6844457809918496
MAE: 0.09752794676806081


In [100]:
result = model.predict(real_test)
output = encoded_test
output['prediction']=result
output_guard = output[output.Pos_G==1].sort_values('prediction',ascending=False)
output_forward = output[output.Pos_F==1].sort_values('prediction',ascending=False)
output_center = output[output.Pos_C==1].sort_values('prediction',ascending=False)

In [101]:
first = pd.concat([output_guard.iloc[0:2], output_forward.iloc[0:2], output_center.iloc[0:1]])
second = pd.concat([output_guard.iloc[2:4], output_forward.iloc[2:4], output_center.iloc[1:2]])
third = pd.concat([output_guard.iloc[4:6], output_forward.iloc[4:6], output_center.iloc[2:3]])

In [102]:
first

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
4,1T,0.952,Luka Doncic,22,65,35.4,28.4,9.1,8.7,1.2,...,3.8,6.4,1.8,8.2,5.9,50.7,0,0,1,0.63186
19,3T,0.220,Trae Young,23,76,34.9,28.4,3.7,9.7,0.9,...,1.0,7.1,-2.0,5.2,4.8,50.0,0,0,1,0.41580
0,1T,1.000,Giannis Antetokounmpo,27,67,32.9,29.9,11.6,5.8,1.1,...,3.7,7.6,3.5,11.2,7.4,71.4,0,1,0,0.94250
9,1T,0.780,Jayson Tatum,23,76,35.9,26.9,8.0,4.4,1.0,...,4.6,4.3,0.6,4.9,4.8,68.1,0,1,0,0.34617
3,1T,0.952,Nikola Jokic,26,74,33.5,27.1,13.8,7.9,1.5,...,4.5,9.2,4.5,13.7,9.8,66.2,1,0,0,0.92239


In [21]:
second

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
6,3T,0.228,Chris Paul,36,65,32.9,14.7,4.4,10.8,1.9,...,3.6,3.1,2.3,5.4,4.0,54.3,0,0,1,0.32342
26,ORV,0.002,Dejounte Murray,25,68,34.8,21.1,8.3,9.2,2.0,...,3.4,3.7,1.7,5.4,4.4,25.7,0,0,1,0.28828
18,3T,0.338,LeBron James,37,56,37.2,30.3,8.2,6.2,1.3,...,2.3,6.9,0.8,7.7,5.1,47.9,0,1,0,0.36780
9,1T,0.780,Jayson Tatum,23,76,35.9,26.9,8.0,4.4,1.0,...,4.6,4.3,0.6,4.9,4.8,68.1,0,1,0,0.35620
8,2T,0.828,Joel Embiid,27,68,33.8,30.6,11.7,4.2,1.1,...,4.1,7.2,2.0,9.2,6.5,68.6,1,0,0,0.91734


In [22]:
third

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
11,2T,0.602,Ja Morant,22,57,33.1,27.4,5.7,6.7,1.2,...,2.1,6.2,-0.1,6.1,3.9,61.4,0,0,1,0.28059
15,2T,0.548,Stephen Curry,33,64,34.5,25.5,5.2,6.3,1.3,...,3.4,5.4,0.4,5.8,4.4,50.0,0,0,1,0.23225
24,ORV,0.064,Jimmy Butler,32,57,33.9,21.4,5.9,5.5,1.6,...,2.9,4.5,1.9,6.3,4.0,52.8,0,1,0,0.29436
16,2T,0.368,DeMar DeRozan,32,76,36.1,27.9,5.2,4.9,0.9,...,1.8,3.4,-0.9,2.5,3.1,47.1,0,1,0,0.20950
22,ORV,0.090,Rudy Gobert,29,66,32.1,15.6,14.7,1.1,0.7,...,4.3,2.9,1.7,4.6,3.6,48.6,1,0,0,0.35723


In [103]:
result = model.predict(real_test2023)
output = encoded_test2023
output['prediction']=result
output_guard = output[output.Pos_G==1].sort_values('prediction',ascending=False)
output_forward = output[output.Pos_F==1].sort_values('prediction',ascending=False)
output_center = output[output.Pos_C==1].sort_values('prediction',ascending=False)
first = pd.concat([output_guard.iloc[0:2], output_forward.iloc[0:2], output_center.iloc[0:1]])
second = pd.concat([output_guard.iloc[2:4], output_forward.iloc[2:4], output_center.iloc[1:2]])
third = pd.concat([output_guard.iloc[4:6], output_forward.iloc[4:6], output_center.iloc[2:3]])

In [104]:
first

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
358,Luka Dončić,23,57,2068,33.0,8.6,8.0,1.5,0.5,0.500,...,7.9,1.6,9.5,6.0,50.7,Dallas Mavericks,0,0,1,0.80293
93,Damian Lillard,32,56,2034,32.4,4.7,7.2,0.9,0.3,0.467,...,8.5,-1.2,7.3,4.8,44.3,Portland Trail Blazers,0,0,1,0.52982
180,Giannis Antetokounmpo,28,55,1784,31.4,11.9,5.5,0.7,0.8,0.542,...,5.6,2.5,8.0,4.5,71.4,Milwaukee Bucks,0,1,0,0.62281
250,Jayson Tatum,24,67,2508,30.0,8.9,4.7,1.0,0.7,0.457,...,4.6,0.4,5.0,4.4,68.1,Boston Celtics,0,1,0,0.51512
422,Nikola Jokić,27,63,2136,24.8,11.8,9.9,1.2,0.7,0.633,...,8.6,4.3,12.9,8.0,66.7,Denver Nuggets,1,0,0,0.83740


In [105]:
second

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
493,Shai Gilgeous-Alexander,24,59,2100,31.3,4.8,5.5,1.7,1.0,0.511,...,5.8,1.5,7.3,4.9,49.3,Oklahoma City Thunder,0,0,1,0.49176
141,Donovan Mitchell,26,61,2165,27.4,4.1,4.6,1.5,0.4,0.477,...,5.3,0.7,6.0,4.4,61.6,Cleveland Cavaliers,0,0,1,0.48876
259,Jimmy Butler,33,57,1897,22.6,6.1,5.0,1.9,0.4,0.527,...,6.4,2.0,8.4,5.0,53.4,Miami Heat,0,1,0,0.46223
353,LeBron James,38,47,1695,29.5,8.4,6.9,0.9,0.6,0.501,...,5.6,0.7,6.3,3.5,47.9,Los Angeles Lakers,0,1,0,0.24597
264,Joel Embiid,28,57,1986,33.5,10.2,4.2,1.1,1.7,0.546,...,6.8,2.3,9.1,5.6,68.6,Philadelphia 76ers,1,0,0,0.81113


In [89]:
# create an XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)

# train the model
model.fit(X_train, y_train)

# make predictions on the test set
y_pred = model.predict(X_test)

# evaluate the model using RMSE, R-squared, and MAE
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('RMSE:', rmse)
print('R-squared:', r2)
print('MAE:', mae)

RMSE: 0.14512146011389204
R-squared: 0.7153915294312762
MAE: 0.08907008007125834


In [90]:
result2 = model.predict(real_test)
output2 = encoded_test
output2['prediction']=result2
output_guard2 = output2[output2.Pos_G==1].sort_values('prediction',ascending=False)
output_forward2 = output2[output2.Pos_F==1].sort_values('prediction',ascending=False)
output_center2 = output2[output2.Pos_C==1].sort_values('prediction',ascending=False)

In [91]:
first2 = pd.concat([output_guard2.iloc[0:2], output_forward2.iloc[0:2], output_center2.iloc[0:1]])
second2 = pd.concat([output_guard2.iloc[2:4], output_forward2.iloc[2:4], output_center2.iloc[1:2]])
third2 = pd.concat([output_guard2.iloc[4:6], output_forward2.iloc[4:6], output_center2.iloc[2:3]])

In [92]:
first2

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
4,1T,0.952,Luka Doncic,22,65,35.4,28.4,9.1,8.7,1.2,...,3.8,6.4,1.8,8.2,5.9,50.7,0,0,1,0.725553
6,3T,0.228,Chris Paul,36,65,32.9,14.7,4.4,10.8,1.9,...,3.6,3.1,2.3,5.4,4.0,54.3,0,0,1,0.476258
0,1T,1.000,Giannis Antetokounmpo,27,67,32.9,29.9,11.6,5.8,1.1,...,3.7,7.6,3.5,11.2,7.4,71.4,0,1,0,0.901661
9,1T,0.780,Jayson Tatum,23,76,35.9,26.9,8.0,4.4,1.0,...,4.6,4.3,0.6,4.9,4.8,68.1,0,1,0,0.478910
3,1T,0.952,Nikola Jokic,26,74,33.5,27.1,13.8,7.9,1.5,...,4.5,9.2,4.5,13.7,9.8,66.2,1,0,0,1.022985


In [93]:
second2

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
5,1T,0.920,Devin Booker,25,68,34.5,26.8,5.0,4.8,1.1,...,3.3,3.9,0.2,4.1,3.6,54.3,0,0,1,0.421475
26,ORV,0.002,Dejounte Murray,25,68,34.8,21.1,8.3,9.2,2.0,...,3.4,3.7,1.7,5.4,4.4,25.7,0,0,1,0.316207
18,3T,0.338,LeBron James,37,56,37.2,30.3,8.2,6.2,1.3,...,2.3,6.9,0.8,7.7,5.1,47.9,0,1,0,0.432400
13,2T,0.552,Kevin Durant,33,55,37.2,29.9,7.4,6.4,0.9,...,2.0,6.4,0.7,7.2,4.8,55.7,0,1,0,0.270153
8,2T,0.828,Joel Embiid,27,68,33.8,30.6,11.7,4.2,1.1,...,4.1,7.2,2.0,9.2,6.5,68.6,1,0,0,0.742659


In [94]:
third2

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
19,3T,0.220,Trae Young,23,76,34.9,28.4,3.7,9.7,0.9,...,1.0,7.1,-2.0,5.2,4.8,50.0,0,0,1,0.280195
15,2T,0.548,Stephen Curry,33,64,34.5,25.5,5.2,6.3,1.3,...,3.4,5.4,0.4,5.8,4.4,50.0,0,0,1,0.261209
24,ORV,0.064,Jimmy Butler,32,57,33.9,21.4,5.9,5.5,1.6,...,2.9,4.5,1.9,6.3,4.0,52.8,0,1,0,0.245640
10,ORV,0.006,Jaylen Brown,25,66,33.6,23.6,6.1,3.5,1.1,...,3.4,1.9,0.1,2.1,2.2,68.1,0,1,0,0.220218
22,ORV,0.090,Rudy Gobert,29,66,32.1,15.6,14.7,1.1,0.7,...,4.3,2.9,1.7,4.6,3.6,48.6,1,0,0,0.219963


In [95]:
result2 = model.predict(real_test2023)
output2 = encoded_test2023
output2['prediction']=result2
output_guard2 = output2[output2.Pos_G==1].sort_values('prediction',ascending=False)
output_forward2 = output2[output2.Pos_F==1].sort_values('prediction',ascending=False)
output_center2 = output2[output2.Pos_C==1].sort_values('prediction',ascending=False)
first2 = pd.concat([output_guard2.iloc[0:2], output_forward2.iloc[0:2], output_center2.iloc[0:1]])
second2 = pd.concat([output_guard2.iloc[2:4], output_forward2.iloc[2:4], output_center2.iloc[1:2]])
third2 = pd.concat([output_guard2.iloc[4:6], output_forward2.iloc[4:6], output_center2.iloc[2:3]])

In [96]:
first2

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
358,Luka Dončić,23,57,2068,33.0,8.6,8.0,1.5,0.5,0.500,...,7.9,1.6,9.5,6.0,50.7,Dallas Mavericks,0,0,1,0.851796
93,Damian Lillard,32,56,2034,32.4,4.7,7.2,0.9,0.3,0.467,...,8.5,-1.2,7.3,4.8,44.3,Portland Trail Blazers,0,0,1,0.653091
180,Giannis Antetokounmpo,28,55,1784,31.4,11.9,5.5,0.7,0.8,0.542,...,5.6,2.5,8.0,4.5,71.4,Milwaukee Bucks,0,1,0,0.607397
250,Jayson Tatum,24,67,2508,30.0,8.9,4.7,1.0,0.7,0.457,...,4.6,0.4,5.0,4.4,68.1,Boston Celtics,0,1,0,0.435433
422,Nikola Jokić,27,63,2136,24.8,11.8,9.9,1.2,0.7,0.633,...,8.6,4.3,12.9,8.0,66.7,Denver Nuggets,1,0,0,0.860513


In [97]:
second2

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
230,James Harden,33,52,1911,21.7,6.3,10.8,1.2,0.5,0.446,...,5.7,0.4,6.1,3.9,68.6,Philadelphia 76ers,0,0,1,0.449351
141,Donovan Mitchell,26,61,2165,27.4,4.1,4.6,1.5,0.4,0.477,...,5.3,0.7,6.0,4.4,61.6,Cleveland Cavaliers,0,0,1,0.397958
259,Jimmy Butler,33,57,1897,22.6,6.1,5.0,1.9,0.4,0.527,...,6.4,2.0,8.4,5.0,53.4,Miami Heat,0,1,0,0.374561
247,Jaylen Brown,26,60,2170,26.7,6.9,3.4,1.2,0.4,0.490,...,1.7,-0.3,1.5,1.9,68.1,Boston Celtics,0,1,0,0.343140
264,Joel Embiid,28,57,1986,33.5,10.2,4.2,1.1,1.7,0.546,...,6.8,2.3,9.1,5.6,68.6,Philadelphia 76ers,1,0,0,0.763704


In [98]:
third2

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
500,Stephen Curry,34,46,1597,29.8,6.2,6.2,0.9,0.3,0.499,...,7.8,0.1,7.9,4.0,50.0,Golden State Warriors,0,0,1,0.387093
493,Shai Gilgeous-Alexander,24,59,2100,31.3,4.8,5.5,1.7,1.0,0.511,...,5.8,1.5,7.3,4.9,49.3,Oklahoma City Thunder,0,0,1,0.364774
353,LeBron James,38,47,1695,29.5,8.4,6.9,0.9,0.6,0.501,...,5.6,0.7,6.3,3.5,47.9,Los Angeles Lakers,0,1,0,0.242062
299,Julius Randle,28,72,2586,25.2,10.3,4.1,0.7,0.3,0.457,...,4.0,-0.1,3.9,3.8,58.3,New York Knicks,0,1,0,0.168538
139,Domantas Sabonis,26,68,2360,19.2,12.5,7.2,0.8,0.5,0.612,...,4.1,1.9,5.9,4.7,61.4,Sacramento Kings,1,0,0,0.359269


In [79]:
# create a CatBoost model
model = CatBoostRegressor(iterations=100, loss_function='RMSE')

# train the model
model.fit(X_train, y_train, verbose=False)

# make predictions on the test set
y_pred = model.predict(X_test)

# evaluate the model using RMSE, R-squared, and MAE
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('RMSE:', rmse)
print('R-squared:', r2)
print('MAE:', mae)

RMSE: 0.1488107834463109
R-squared: 0.700736776837528
MAE: 0.09661202487237971


In [80]:
result3 = model.predict(real_test)
output3 = encoded_test
output3['prediction']=result3
output_guard3 = output3[output3.Pos_G==1].sort_values('prediction',ascending=False)
output_forward3 = output3[output3.Pos_F==1].sort_values('prediction',ascending=False)
output_center3 = output3[output3.Pos_C==1].sort_values('prediction',ascending=False)

In [81]:
first3 = pd.concat([output_guard3.iloc[0:2], output_forward3.iloc[0:2], output_center3.iloc[0:1]])
second3 = pd.concat([output_guard3.iloc[2:4], output_forward3.iloc[2:4], output_center3.iloc[1:2]])
third3 = pd.concat([output_guard3.iloc[4:6], output_forward3.iloc[4:6], output_center3.iloc[2:3]])

In [82]:
first3

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
4,1T,0.952,Luka Doncic,22,65,35.4,28.4,9.1,8.7,1.2,...,3.8,6.4,1.8,8.2,5.9,50.7,0,0,1,0.638953
19,3T,0.220,Trae Young,23,76,34.9,28.4,3.7,9.7,0.9,...,1.0,7.1,-2.0,5.2,4.8,50.0,0,0,1,0.482165
0,1T,1.000,Giannis Antetokounmpo,27,67,32.9,29.9,11.6,5.8,1.1,...,3.7,7.6,3.5,11.2,7.4,71.4,0,1,0,1.032679
18,3T,0.338,LeBron James,37,56,37.2,30.3,8.2,6.2,1.3,...,2.3,6.9,0.8,7.7,5.1,47.9,0,1,0,0.588218
8,2T,0.828,Joel Embiid,27,68,33.8,30.6,11.7,4.2,1.1,...,4.1,7.2,2.0,9.2,6.5,68.6,1,0,0,0.899044


In [83]:
second3

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
11,2T,0.602,Ja Morant,22,57,33.1,27.4,5.7,6.7,1.2,...,2.1,6.2,-0.1,6.1,3.9,61.4,0,0,1,0.419289
15,2T,0.548,Stephen Curry,33,64,34.5,25.5,5.2,6.3,1.3,...,3.4,5.4,0.4,5.8,4.4,50.0,0,0,1,0.330783
9,1T,0.780,Jayson Tatum,23,76,35.9,26.9,8.0,4.4,1.0,...,4.6,4.3,0.6,4.9,4.8,68.1,0,1,0,0.462691
13,2T,0.552,Kevin Durant,33,55,37.2,29.9,7.4,6.4,0.9,...,2.0,6.4,0.7,7.2,4.8,55.7,0,1,0,0.394127
3,1T,0.952,Nikola Jokic,26,74,33.5,27.1,13.8,7.9,1.5,...,4.5,9.2,4.5,13.7,9.8,66.2,1,0,0,0.798732


In [84]:
third3

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
6,3T,0.228,Chris Paul,36,65,32.9,14.7,4.4,10.8,1.9,...,3.6,3.1,2.3,5.4,4.0,54.3,0,0,1,0.239774
23,ORV,0.008,Donovan Mitchell,25,67,33.8,25.9,4.2,5.3,1.5,...,2.5,4.6,-0.3,4.3,3.6,48.6,0,0,1,0.189189
16,2T,0.368,DeMar DeRozan,32,76,36.1,27.9,5.2,4.9,0.9,...,1.8,3.4,-0.9,2.5,3.1,47.1,0,1,0,0.342510
10,ORV,0.006,Jaylen Brown,25,66,33.6,23.6,6.1,3.5,1.1,...,3.4,1.9,0.1,2.1,2.2,68.1,0,1,0,0.176288
17,3T,0.348,Karl-Anthony Towns,26,74,33.5,24.6,9.8,3.6,1.0,...,3.2,4.5,0.5,5.0,4.4,48.6,1,0,0,0.321740


In [85]:
result3 = model.predict(real_test2023)
output3 = encoded_test2023
output3['prediction']=result3
output_guard3 = output3[output3.Pos_G==1].sort_values('prediction',ascending=False)
output_forward3 = output3[output3.Pos_F==1].sort_values('prediction',ascending=False)
output_center3 = output3[output3.Pos_C==1].sort_values('prediction',ascending=False)
first3 = pd.concat([output_guard3.iloc[0:2], output_forward3.iloc[0:2], output_center3.iloc[0:1]])
second3 = pd.concat([output_guard3.iloc[2:4], output_forward3.iloc[2:4], output_center3.iloc[1:2]])
third3 = pd.concat([output_guard3.iloc[4:6], output_forward3.iloc[4:6], output_center3.iloc[2:3]])

In [86]:
first3

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
493,Shai Gilgeous-Alexander,24,59,2100,31.3,4.8,5.5,1.7,1.0,0.511,...,5.8,1.5,7.3,4.9,49.3,Oklahoma City Thunder,0,0,1,0.639240
358,Luka Dončić,23,57,2068,33.0,8.6,8.0,1.5,0.5,0.500,...,7.9,1.6,9.5,6.0,50.7,Dallas Mavericks,0,0,1,0.573898
259,Jimmy Butler,33,57,1897,22.6,6.1,5.0,1.9,0.4,0.527,...,6.4,2.0,8.4,5.0,53.4,Miami Heat,0,1,0,0.540526
250,Jayson Tatum,24,67,2508,30.0,8.9,4.7,1.0,0.7,0.457,...,4.6,0.4,5.0,4.4,68.1,Boston Celtics,0,1,0,0.497656
264,Joel Embiid,28,57,1986,33.5,10.2,4.2,1.1,1.7,0.546,...,6.8,2.3,9.1,5.6,68.6,Philadelphia 76ers,1,0,0,1.000947


In [87]:
second3

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
230,James Harden,33,52,1911,21.7,6.3,10.8,1.2,0.5,0.446,...,5.7,0.4,6.1,3.9,68.6,Philadelphia 76ers,0,0,1,0.438289
205,Ja Morant,23,53,1721,27.1,6.0,8.2,1.1,0.2,0.463,...,5.6,0.7,6.3,3.6,61.4,Memphis Grizzlies,0,0,1,0.408836
180,Giannis Antetokounmpo,28,55,1784,31.4,11.9,5.5,0.7,0.8,0.542,...,5.6,2.5,8.0,4.5,71.4,Milwaukee Bucks,0,1,0,0.454713
353,LeBron James,38,47,1695,29.5,8.4,6.9,0.9,0.6,0.501,...,5.6,0.7,6.3,3.5,47.9,Los Angeles Lakers,0,1,0,0.329275
422,Nikola Jokić,27,63,2136,24.8,11.8,9.9,1.2,0.7,0.633,...,8.6,4.3,12.9,8.0,66.7,Denver Nuggets,1,0,0,0.815934


In [88]:
third3

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
141,Donovan Mitchell,26,61,2165,27.4,4.1,4.6,1.5,0.4,0.477,...,5.3,0.7,6.0,4.4,61.6,Cleveland Cavaliers,0,0,1,0.375353
93,Damian Lillard,32,56,2034,32.4,4.7,7.2,0.9,0.3,0.467,...,8.5,-1.2,7.3,4.8,44.3,Portland Trail Blazers,0,0,1,0.343807
299,Julius Randle,28,72,2586,25.2,10.3,4.1,0.7,0.3,0.457,...,4.0,-0.1,3.9,3.8,58.3,New York Knicks,0,1,0,0.187098
247,Jaylen Brown,26,60,2170,26.7,6.9,3.4,1.2,0.4,0.490,...,1.7,-0.3,1.5,1.9,68.1,Boston Celtics,0,1,0,0.172701
139,Domantas Sabonis,26,68,2360,19.2,12.5,7.2,0.8,0.5,0.612,...,4.1,1.9,5.9,4.7,61.4,Sacramento Kings,1,0,0,0.277660


In [35]:
# create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
# set the hyperparameters for the model
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',

}
# train the model
num_rounds = 100
model = lgb.train(params, train_data, num_rounds)

# make predictions on the test set
y_pred = model.predict(X_test)
# evaluate the model using root mean squared error, R-squared, and mean absolute error
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('RMSE:', rmse)
print('R-squared:', r2)
print('MAE:', mae)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000836 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3975
[LightGBM] [Info] Number of data points in the train set: 1049, number of used features: 32
[LightGBM] [Info] Start training from score 0.235592
RMSE: 0.13938001804047734
R-squared: 0.7374659878622984
MAE: 0.0880323990641525


In [36]:
result7 = model.predict(real_test)
output7 = encoded_test
output7['prediction']=result7
output_guard7 = output7[output7.Pos_G==1].sort_values('prediction',ascending=False)
output_forward7 = output7[output7.Pos_F==1].sort_values('prediction',ascending=False)
output_center7 = output7[output7.Pos_C==1].sort_values('prediction',ascending=False)

In [37]:
first7 = pd.concat([output_guard7.iloc[0:2], output_forward7.iloc[0:2], output_center7.iloc[0:1]])
second7 = pd.concat([output_guard7.iloc[2:4], output_forward7.iloc[2:4], output_center7.iloc[1:2]])
third7 = pd.concat([output_guard7.iloc[4:6], output_forward7.iloc[4:6], output_center7.iloc[2:3]])

In [38]:
first7

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
4,1T,0.952,Luka Doncic,22,65,35.4,28.4,9.1,8.7,1.2,...,3.8,6.4,1.8,8.2,5.9,50.7,0,0,1,0.727464
6,3T,0.228,Chris Paul,36,65,32.9,14.7,4.4,10.8,1.9,...,3.6,3.1,2.3,5.4,4.0,54.3,0,0,1,0.550673
0,1T,1.000,Giannis Antetokounmpo,27,67,32.9,29.9,11.6,5.8,1.1,...,3.7,7.6,3.5,11.2,7.4,71.4,0,1,0,0.928300
9,1T,0.780,Jayson Tatum,23,76,35.9,26.9,8.0,4.4,1.0,...,4.6,4.3,0.6,4.9,4.8,68.1,0,1,0,0.487433
3,1T,0.952,Nikola Jokic,26,74,33.5,27.1,13.8,7.9,1.5,...,4.5,9.2,4.5,13.7,9.8,66.2,1,0,0,1.080781


In [39]:
second7

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
19,3T,0.220,Trae Young,23,76,34.9,28.4,3.7,9.7,0.9,...,1.0,7.1,-2.0,5.2,4.8,50.0,0,0,1,0.384455
11,2T,0.602,Ja Morant,22,57,33.1,27.4,5.7,6.7,1.2,...,2.1,6.2,-0.1,6.1,3.9,61.4,0,0,1,0.377262
18,3T,0.338,LeBron James,37,56,37.2,30.3,8.2,6.2,1.3,...,2.3,6.9,0.8,7.7,5.1,47.9,0,1,0,0.426151
16,2T,0.368,DeMar DeRozan,32,76,36.1,27.9,5.2,4.9,0.9,...,1.8,3.4,-0.9,2.5,3.1,47.1,0,1,0,0.314091
8,2T,0.828,Joel Embiid,27,68,33.8,30.6,11.7,4.2,1.1,...,4.1,7.2,2.0,9.2,6.5,68.6,1,0,0,0.815182


In [40]:
third7

,# Tm,Share,Player,Age,G,MP,PTS,TRB,AST,STL,...,DWS,OBPM,DBPM,BPM,VORP,WinPCT,Pos_C,Pos_F,Pos_G,prediction
5,1T,0.920,Devin Booker,25,68,34.5,26.8,5.0,4.8,1.1,...,3.3,3.9,0.2,4.1,3.6,54.3,0,0,1,0.364179
26,ORV,0.002,Dejounte Murray,25,68,34.8,21.1,8.3,9.2,2.0,...,3.4,3.7,1.7,5.4,4.4,25.7,0,0,1,0.258050
13,2T,0.552,Kevin Durant,33,55,37.2,29.9,7.4,6.4,0.9,...,2.0,6.4,0.7,7.2,4.8,55.7,0,1,0,0.260196
24,ORV,0.064,Jimmy Butler,32,57,33.9,21.4,5.9,5.5,1.6,...,2.9,4.5,1.9,6.3,4.0,52.8,0,1,0,0.218054
17,3T,0.348,Karl-Anthony Towns,26,74,33.5,24.6,9.8,3.6,1.0,...,3.2,4.5,0.5,5.0,4.4,48.6,1,0,0,0.403527


In [69]:
result7 = model.predict(real_test2023)
output7 = encoded_test2023
output7['prediction']=result7
output_guard7 = output7[output7.Pos_G==1].sort_values('prediction',ascending=False)
output_forward7 = output7[output7.Pos_F==1].sort_values('prediction',ascending=False)
output_center7 = output7[output7.Pos_C==1].sort_values('prediction',ascending=False)

In [70]:
first7 = pd.concat([output_guard7.iloc[0:2], output_forward7.iloc[0:2], output_center7.iloc[0:1]])
second7 = pd.concat([output_guard7.iloc[2:4], output_forward7.iloc[2:4], output_center7.iloc[1:2]])
third7 = pd.concat([output_guard7.iloc[4:6], output_forward7.iloc[4:6], output_center7.iloc[2:3]])

In [71]:
first7

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
358,Luka Dončić,23,57,2068,33.0,8.6,8.0,1.5,0.5,0.500,...,7.9,1.6,9.5,6.0,50.7,Dallas Mavericks,0,0,1,0.911283
493,Shai Gilgeous-Alexander,24,59,2100,31.3,4.8,5.5,1.7,1.0,0.511,...,5.8,1.5,7.3,4.9,49.3,Oklahoma City Thunder,0,0,1,0.542732
180,Giannis Antetokounmpo,28,55,1784,31.4,11.9,5.5,0.7,0.8,0.542,...,5.6,2.5,8.0,4.5,71.4,Milwaukee Bucks,0,1,0,0.667168
250,Jayson Tatum,24,67,2508,30.0,8.9,4.7,1.0,0.7,0.457,...,4.6,0.4,5.0,4.4,68.1,Boston Celtics,0,1,0,0.504450
422,Nikola Jokić,27,63,2136,24.8,11.8,9.9,1.2,0.7,0.633,...,8.6,4.3,12.9,8.0,66.7,Denver Nuggets,1,0,0,0.884223


In [72]:
second7

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
205,Ja Morant,23,53,1721,27.1,6.0,8.2,1.1,0.2,0.463,...,5.6,0.7,6.3,3.6,61.4,Memphis Grizzlies,0,0,1,0.467686
141,Donovan Mitchell,26,61,2165,27.4,4.1,4.6,1.5,0.4,0.477,...,5.3,0.7,6.0,4.4,61.6,Cleveland Cavaliers,0,0,1,0.381192
259,Jimmy Butler,33,57,1897,22.6,6.1,5.0,1.9,0.4,0.527,...,6.4,2.0,8.4,5.0,53.4,Miami Heat,0,1,0,0.260337
299,Julius Randle,28,72,2586,25.2,10.3,4.1,0.7,0.3,0.457,...,4.0,-0.1,3.9,3.8,58.3,New York Knicks,0,1,0,0.206971
264,Joel Embiid,28,57,1986,33.5,10.2,4.2,1.1,1.7,0.546,...,6.8,2.3,9.1,5.6,68.6,Philadelphia 76ers,1,0,0,0.785940


In [73]:
third7

,Player,Age,G,MP,PTS,TRB,AST,STL,BLK,FG%,...,OBPM,DBPM,BPM,VORP,WinPCT,Team,Pos_C,Pos_F,Pos_G,prediction
93,Damian Lillard,32,56,2034,32.4,4.7,7.2,0.9,0.3,0.467,...,8.5,-1.2,7.3,4.8,44.3,Portland Trail Blazers,0,0,1,0.357382
230,James Harden,33,52,1911,21.7,6.3,10.8,1.2,0.5,0.446,...,5.7,0.4,6.1,3.9,68.6,Philadelphia 76ers,0,0,1,0.331131
353,LeBron James,38,47,1695,29.5,8.4,6.9,0.9,0.6,0.501,...,5.6,0.7,6.3,3.5,47.9,Los Angeles Lakers,0,1,0,0.167448
122,DeMar DeRozan,33,64,2343,25.4,4.7,5.1,1.1,0.5,0.513,...,2.0,0.2,2.2,2.4,47.1,Chicago Bulls,0,1,0,0.087599
139,Domantas Sabonis,26,68,2360,19.2,12.5,7.2,0.8,0.5,0.612,...,4.1,1.9,5.9,4.7,61.4,Sacramento Kings,1,0,0,0.359562
